## 정보검색
1. Vector Space Model
2. Indexing
3. Inverted Document
4. Weighting
5. Retrieving


- TF/IDF
![7](https://user-images.githubusercontent.com/38183218/43502447-e3624b86-9595-11e8-895b-150ed944052c.jpg)

- Similarity
![8](https://user-images.githubusercontent.com/38183218/43502459-f96733f6-9595-11e8-8094-60a1791d8daa.jpg)
![9](https://user-images.githubusercontent.com/38183218/43502460-f991e916-9595-11e8-880c-6295e33fdf91.jpg)

실습: 뉴스doc을 구축해보자

In [85]:
import requests
from bs4 import BeautifulSoup
import glob
import re
import nltk
from konlpy.tag import Kkma

In [2]:
resp = requests.get("http://media.daum.net")

In [6]:
soup = BeautifulSoup(resp.content,'lxml')

In [15]:
aList = soup.select('.list_headline  a.link_txt')

In [20]:
links = []
for a in aList:
    links.append(a.get('href'))

In [21]:
links

['http://v.media.daum.net/v/20180801141138247',
 'http://v.media.daum.net/v/20180801140951174',
 'http://v.media.daum.net/v/20180801140921160',
 'http://v.media.daum.net/v/20180801140802122',
 'http://v.media.daum.net/v/20180801140622059',
 'http://v.media.daum.net/v/20180801140406973',
 'http://v.media.daum.net/v/20180801140352961',
 'http://v.media.daum.net/v/20180801140009800',
 'http://v.media.daum.net/v/20180801140007797',
 'http://v.media.daum.net/v/20180801140007793',
 'http://v.media.daum.net/v/20180801140004784',
 'http://v.media.daum.net/v/20180801135922758']

In [39]:
news_docs = []
for link in links:
    resp = requests.get(link)
    news_soup = BeautifulSoup(resp.content,'lxml')
    
    title = news_soup.find('title')
    pars = news_soup.select('.article_view p ')
    article = ''.join([par.text for par in pars if len(par)>0])
    news_docs.append((title.text, article))

In [58]:
news_docs[0]

('강원도 홍천 40.3도..우리나라 기상관측 이래 역대 최고(2보) | Daum 뉴스',
 '(서울=연합뉴스) 김승욱 기자 = 강원도 홍천의 수은주가 1일 40.3도까지 치솟아 기상관측 이래 역대 최고기온을 갈아치웠다.기상청에 따르면 이날 오후 1시 59시께 40.1도를 기록한 뒤 오후 2시 1분께 40.3도로 기온이 더 올랐다.우리나라 기상관측 역대 최고 온도다.부산·인천 1904년, 서울 1907년 등 현대적인 기상관측 장비가 도입된 20세기 초반 이래 전국에서 40도를 돌파한 적은 1942년 8월 1일 대구(40.0도)가 유일했다.앞서 경북 의성은 올해 7월 27일 39.9도, 충북 추풍령은 1939년 7월 21일 39.8도를 기록한 바 있다.ksw08@yna.co.kr')

In [41]:
len(news_docs)

12

In [52]:
for i in range(len(news_docs)):
    with open('docs/news_docs{0}.txt'.format(i),'w',encoding='utf-8') as fp:
        fp.write(news_docs[i][0]+'\n'+news_docs[i][1])

In [69]:
fileList = glob.glob('/Users/Chankoo/Desktop/GitHub/python/docs/*')

In [112]:
for file in fileList[:1]: 
    with open(file,'r',encoding='utf-8') as fp:
        content = fp.read()
        content = re.sub(r"[\s]{2,}","",content) # strip
        for sentence in nltk.sent_tokenize(content):
            words = nltk.regexp_tokenize(sentence, r"[0-9ㄱ-ㅎㅏ-ㅣ가-힛]+") # 한글 tokenizing을 위해 regexp를 쓴다
            for word in words:
                tokens = []
                for i in range(len(word)):
                    if i+1<len(word): # 1음절, 마지막음절 제거
                        tokens.append(word[i:i+2]) # bigram으로 자름
                print(word)
                print(tokens, end='\n')

강원도
['강원', '원도']
홍천
['홍천']
40
['40']
3도
['3도']
우리나라
['우리', '리나', '나라']
기상관측
['기상', '상관', '관측']
이래
['이래']
역대
['역대']
최고
['최고']
2보
['2보']
뉴스
['뉴스']
서울
['서울']
연합뉴스
['연합', '합뉴', '뉴스']
김승욱
['김승', '승욱']
기자
['기자']
강원도
['강원', '원도']
홍천의
['홍천', '천의']
수은주가
['수은', '은주', '주가']
1일
['1일']
40
['40']
3도까지
['3도', '도까', '까지']
치솟아
['치솟', '솟아']
기상관측
['기상', '상관', '관측']
이래
['이래']
역대
['역대']
최고기온을
['최고', '고기', '기온', '온을']
갈아치웠다
['갈아', '아치', '치웠', '웠다']
기상청에
['기상', '상청', '청에']
따르면
['따르', '르면']
이날
['이날']
오후
['오후']
1시
['1시']
59시께
['59', '9시', '시께']
40
['40']
1도를
['1도', '도를']
기록한
['기록', '록한']
뒤
[]
오후
['오후']
2시
['2시']
1분께
['1분', '분께']
40
['40']
3도로
['3도', '도로']
기온이
['기온', '온이']
더
[]
올랐다
['올랐', '랐다']
우리나라
['우리', '리나', '나라']
기상관측
['기상', '상관', '관측']
역대
['역대']
최고
['최고']
온도다
['온도', '도다']
부산
['부산']
인천
['인천']
1904년
['19', '90', '04', '4년']
서울
['서울']
1907년
['19', '90', '07', '7년']
등
[]
현대적인
['현대', '대적', '적인']
기상관측
['기상', '상관', '관측']
장비가
['장비', '비가']
도입된
['도입', '입된']
20세기
['20', '0세', '세기']
초반
['초반']
이래
['이래']
전국에서
['전국', '국

In [110]:
unq_nouns = [] # vetorize를 위해 unique한 noun을 뽑아낸다

for sentence in nltk.sent_tokenize(content):
    nouns = []
    for word in nltk.word_tokenize(sentence):
        nouns.extend(Kkma().nouns(word))
        nouns = list(set(nouns))
    unq_nouns.extend(nouns)

unq_nouns = list(set(unq_nouns))
print(len(unq_nouns)) # 차원의 수

185
